### Exercício: Análise Manual do Dataset Disney+ 

**Objetivo:**
Este exercício tem como objetivo desenvolver suas habilidades em manipulação manual de dados em Python, processando um dataset de shows da Disney+ sem o auxílio de bibliotecas
de parsing como csv ou pandas. Você deverá extrair informações relevantes e gerar um relatório detalhado.

**Dataset:**

O dataset a ser utilizado está disponível no seguinte link: https://www.kaggle.com/datasets/eshummalik/disney

**Instruções:**

Download do Dataset: Baixe o arquivo do dataset manualmente a partir do link fornecido. --ok

**Carregamento Manual dos Dados:**

* Implemente uma função chamada abrir_filmes_disney() que será responsável por abrir o arquivo do dataset.--ok

Você não deve utilizar bibliotecas como csv ou pandas para ler o arquivo. A leitura deve ser feita linha por linha, utilizando as funcionalidades básicas de manipulação de arquivos em Python.

* Ignore linhas vazias encontradas no arquivo. --ok
* Para cada linha do dataset, identifique se ela representa um filme (type coluna) ou uma série. --ok

* Crie duas classes em Python: Filme e Serie. Cada linha do dataset deve ser mapeada para uma instância da classe Filme ou Serie correspondente, 
contendo seus respectivos atributos (baseados nas colunas do dataset). Implemente getters e setters para os atributos conforme necessário. --ok
* Os campos que contêm o valor "N/A" no dataset devem ser tratados e armazenados como None nos objetos Filme ou Serie. --ok
* A função abrir_filmes_disney() deve retornar uma lista contendo todos os objetos Filme e Serie criados a partir do dataset. --ok
* Certifique-se de que o arquivo do dataset seja devidamente fechado ao final da execução da função abrir_filmes_disney(). --ok utilizado o with

**Geração do Relatório:**

* Implemente uma função chamada gerar_relatorio() que receberá como entrada a lista de objetos Filme e Serie retornada pela função abrir_filmes_disney(). --ok

**Com base nos dados contidos nos objetos, calcule e inclua no relatório as seguintes informações:**

* A média da nota de todos os filmes no IMDB.
* A média da nota de todos os filmes no Metascore.
* A média do número de votos de todos os filmes no IMDB.
* As 3 línguas mais usadas e as 3 línguas menos usadas no dataset.
* Os 3 atores que mais aparecem e os 3 atores que menos aparecem no dataset.
* O diretor com mais filmes no dataset.
* O diretor com o filme mais popular no IMDB (considerando a maior nota IMDB).
* O diretor com o filme mais popular no Metascore (considerando a maior nota Metascore).
* O ano em que mais filmes foram lançados.
* A pior série segundo a nota IMDB e a pior série segundo a nota Metascore.
* Uma lista de filmes que possuem mais de um lançamento (considerados "remakes" ou diferentes versões no dataset, identificados por títulos iguais mas anos de lançamento diferentes).
* O relatório de saída deve ser salvo em um arquivo texto chamado relatorio-disney.txt no mesmo diretório do script.

**Estrutura do Código: Organize seu código de forma modular, utilizando as funções abrir_filmes_disney() e gerar_relatorio() conforme especificado.Você não é obrigado a passar parâmetros para essas funções, mas pode fazê-lo se julgar necessário para uma melhor organização do código.**

In [ ]:
import os
from functools import reduce
from collections import Counter

# Nome do arquivo do dataset (assumindo que você baixou e salvou)
arquivo_disney = "./arquivo_origem/disney_plus_shows.csv"
arquivo_relatorio = "./arquivo_saida/relatorio-disney.txt"

# *** Total de 19 colunas do arquivo:      
# imdb_id,[0]# title,[1]# plot,[2]# type,[3]# rated,# year,# released_at,# added_at,# runtime,# enre,# director,# writer,# actors,# language,# country,# awards,# metascore,# imdb_rating,# imdb_votes

# Descrição das colunas:
# imdb_id → Código exclusivo do IMDb.
# title → Nome do filme/programa.
# plot → Resumo da história curta.
# type → Filme ou Série.
# rated → Classificação etária (G, PG, PG-13 etc).
# year → Ano de lançamento.
# released_at → Data de lançamento.
# added_at → Data adicionada na plataforma Disney.
# runtime → Duração total.
# genre → Categoria como Comédia, Drama, Animação.
# director → Diretor do programa.
# writer → Escritor de histórias ou roteiros.
# actors → Elenco principal.
# language → Idioma(s) usado(s).

# ***Colunas para a classe Conteudo:
# imdb_id # title # runtime# released# year# genres# metascore# imdb_score# imdb_votes# actors# directors# language 

#Colunas para a classe Filme
#type = "movie"

#Colunas para a classe Série
#type = "series"

# ----------------------------------------------------
#               Classes de Entidade
# ----------------------------------------------------

class Conteudo:
    """Classe base para Filme e Serie com atributos comuns e tratamento N/A."""
    def __init__(self, dados):
      
        # Atributos
        #print(dados)
        self._title = self.trata_na(dados[1])
        self._year = self.trata_na(dados[4], int)
        self._metascore = self.trata_na(dados[15], float) 
        self._imdb_rating = self.trata_na(dados[16], float) 
        self._imdb_votes = self.trata_na(dados[17], float) 
        #print(self._title)
        #print(self._year)
        #print(self._metascore)
        #print(self._imdb_rating)
        #print(self._imdb_votes)
       
        # Atributos de lista/string 
        self._language = self.trata_lista(dados[12]) 
        self._actors = self.trata_lista(dados[11], split_char=',') 
        self._directors = self.trata_lista(dados[9], split_char=',') 

        #print(self._language)
        #print(self._actors)
        #print(self._directors)

    def trata_na(self,value, target_type=str):
        if value is None or value.strip() in ("N/A", ""):
            return None
        
        cleaned_value = value.strip()
        if target_type in (float, int):
            # substituição da vírgula pelo ponto 
            cleaned_value = cleaned_value.replace(",", ".") # por causa do campo imdb_votes que apresenta (.)

        try:
            return target_type(cleaned_value)
        except ValueError:
            return None
                
    def trata_lista(self,value, split_char=','):
        """Trata N/A e strings com múltiplos valores separados por vírgula/outro."""
        if value is None or value.strip() in ("N/A", ""):
            return []
        # Retorna uma lista de strings limpas
        return [item.strip() for item in value.split(split_char) if item.strip()]

    # Getters
    def get_title(self): 
        return self._title
    
    def get_year(self): 
        return self._year
    
    def get_metascore(self): 
        return self._metascore
    
    def get_imdb_rating(self): 
        return self._imdb_rating
    
    def get_imdb_votes(self): 
        return self._imdb_votes
    
    def get_actors(self): 
        return self._actors
    
    def get_directors(self): 
        return self._directors
    
    def get_language(self): 
        return self._language

    # apenas por boa prática)
    def set_title(self, novo_titulo): 
        self._title = novo_titulo

class Filme(Conteudo):
    def __init__(self, dados):
        super().__init__(dados)
        self.type = "movie"

    def __repr__(self):
        return f"Filme(Título: {self._title}, Ano: {self._year}, IMDB: {self._imdb_rating})"


class Serie(Conteudo):
    def __init__(self, dados):
        super().__init__(dados)
        self.type = "series"

    def __repr__(self):
        return f"Série(Título: {self._title}, Ano: {self._year}, IMDB: {self._imdb_rating})"


# ----------------------------------------------------
#            Funções de Manipulação de Dados
# ----------------------------------------------------

def split_csv_manual(linha):
    """Divide uma linha CSV respeitando campos entre aspas."""
    campos = []
    campo_atual = ""
    dentro_de_aspas = False
    
    # Adicionamos uma vírgula no final para garantir que o último campo seja processado
    linha_ajustada = linha.strip() + ","
    #print(linha_ajustada)

    for char in linha_ajustada:
        if char == '"':
            # Alterna o estado de estar dentro ou fora das aspas
            dentro_de_aspas = not dentro_de_aspas
            # Adiciona a aspa ao campo, pois ela deve ser preservada para remoção posterior
            campo_atual += char
        
        elif char == ',' and not dentro_de_aspas:
            # Encontramos o separador real (vírgula fora das aspas)
            
            # Limpa o campo (remove as aspas externas e espaços)
            campo_limpo = campo_atual.strip().strip('"')
            campos.append(campo_limpo)
            
            # Reseta o acumulador do campo
            campo_atual = ""
        
        else:
            # Adiciona o caractere ao campo atual
            campo_atual += char
            
    return campos

def abrir_filmes_disney():
    """
    Carrega o dataset manualmente linha por linha.
    A função abrir_filmes_disney() deve retornar uma lista contendo todos os objetos Filme e Serie criados a partir do dataset
    """
    conteudos = []
    
    try:
        with open(arquivo_disney, 'r', encoding='utf-8') as file:
            try:
                next(file) # Ignora o cabeçalho
               # print ('Arquivo aberto')
            except StopIteration:
                return []
            
            for linha in file:
                if not linha.strip():
                    continue
                
                #dados = [col.strip().strip('"') for col in linha.strip().split(',')]
                #dados_teste = [col.strip().strip('"') for col in linha.strip().split('"')]
                #print(linha)
                dados = split_csv_manual(linha)
                if len(dados) < 18:
                    continue                
                dados.pop(2) #Removendo o campo reumo que atrapalha e não é utilizado
                #print('pop')
                #print(dados)              
                tipo = dados[2].lower() 
                             
                if tipo == 'movie':
                    conteudos.append(Filme(dados))
                    #direciona para filmes
                elif tipo == 'series':
                    conteudos.append(Serie(dados))
                    #direciona para series

    except FileNotFoundError:
        print(f"ERRO: Arquivo '{arquivo_disney}' não encontrado.")
        return []
    except Exception as e:
        print(f"Ocorreu um erro na leitura do arquivo: {e}")
        return []

    return conteudos


def gerar_relatorio(lista_conteudos):
    """
    Gera o relatório de análise e salva em um arquivo de texto.
    """
    if not lista_conteudos:
        return "Nenhum dado para analisar."

    # Separação por tipo Olhando para a classe filha
    filmes = [c for c in lista_conteudos if isinstance(c, Filme)]
    print(filmes)
    series = [c for c in lista_conteudos if isinstance(c, Serie)]
    print(series)

    # Filtra conteúdos válidos para cálculo de médias

    filmes_com_imdb_rating = [item_filme for item_filme in filmes if item_filme.get_imdb_rating() is not None] #elimina None por causa do calculo de média
    print(filmes_com_imdb_rating)
    filmes_com_metascore = [item_filme for item_filme in filmes if item_filme.get_metascore() is not None]
    print(filmes_com_metascore)
    filmes_com_votos = [item_filme for item_filme in filmes if item_filme.get_imdb_votes() is not None]
    print(filmes_com_votos)

    #A pior série segundo a nota IMDB e a pior série segundo a nota Metascore.
    series_com_imdb = [item_serie for item_serie in series if item_serie.get_imdb_rating() is not None]
    print(series_com_imdb)
    series_com_metascore = [item_serie for item_serie in series if item_serie.get_metascore() is not None]
    print(series_com_metascore)

    # --- Cálculos de Média ---

     # if len(filmes_com_imdb_rating) > 0:
    #     acumulador_imdb = 0
    #     for filme in filmes_com_imdb_rating:
    #         acumulador_imdb += filme.get_imdb_rating()
    #     media_imdb = acumulador_imdb / len(filmes_com_imdb_rating)
    # else:
    #     media_imdb = 0.0

    media_imdb = reduce(lambda acumulador_imdb, item_filme: acumulador_imdb + item_filme.get_imdb_rating(), filmes_com_imdb_rating, 0) / len(filmes_com_imdb_rating) if filmes_com_imdb_rating else 0.0 #so aplica a divisao por len(filmes_com_imdb_rating) se o if for true 
    print(media_imdb)
    media_metascore = reduce(lambda acumulador_metascore, item_filme: acumulador_metascore + item_filme.get_metascore(), filmes_com_metascore, 0) / len(filmes_com_metascore) if filmes_com_metascore else 0.0
    print(media_metascore)
    media_votos = reduce(lambda acumulador_votos, item_filme: acumulador_votos + item_filme.get_imdb_votes(), filmes_com_votos, 0) / len(filmes_com_votos) if filmes_com_votos else 0.0
    print(media_votos)

    # --- Contagens (Atores, Diretores, Línguas) ---
   # *todas_linguagens = []
    todas_linguagens = []
    todos_atores = []
    todos_diretores = []
    
    for item in lista_conteudos:
        todas_linguagens.extend(item.get_language()) # extend para retornar lista de strings ['English', 'French', 'English', 'English', 'Spanish', 'English', 'English', 'English']
        # *todas_linguagens.append(item.get_language()) #retorna lista de lista[['English', 'French'], ['English'], ['English', 'Spanish']]
        todos_atores.extend(item.get_actors())
        todos_diretores.extend(item.get_directors())

    #print(todas_linguagens) 

    contagem_linguagens = Counter(todas_linguagens)
    contagem_atores = Counter(todos_atores)
    contagem_diretores = Counter(todos_diretores)

    # --- Análise de Extremos ---
    diretor_mais_filmes = contagem_diretores.most_common(1)[0] if contagem_diretores else ('N/A', 0)
    
    # Ano de maior lançamento
    anos = [c.get_year() for c in lista_conteudos if c.get_year() is not None]
    contagem_anos = Counter(anos)
    ano_mais_filmes = contagem_anos.most_common(1)[0] if contagem_anos else ('N/A', 0)

    # Variáveis de fallback para evitar NoneType
    diretor_popular_imdb = "N/A"; filme_max_imdb_titulo = "N/A"; filme_max_imdb_rating = "N/A"
    diretor_popular_meta = "N/A"; filme_max_meta_titulo = "N/A"; filme_max_meta_score = "N/A"
    pior_serie_imdb_titulo = "N/A"; pior_serie_imdb_rating = "N/A"
    pior_serie_meta_titulo = "N/A"; pior_serie_meta_score = "N/A"

    # Filme mais popular por IMDB 
    filme_max_imdb = max(filmes_com_imdb_rating, key=lambda f: f.get_imdb_rating()) if filmes_com_imdb_rating else None
    if filme_max_imdb:
        filme_max_imdb_rating = f"{filme_max_imdb.get_imdb_rating():.1f}"
        filme_max_imdb_titulo = filme_max_imdb.get_title()
        diretor_popular_imdb = filme_max_imdb.get_directors()[0] if filme_max_imdb.get_directors() else "N/A"

    # Filme mais popular por Metascore 
    filme_max_meta = max(filmes_com_metascore, key=lambda f: f.get_metascore()) if filmes_com_metascore else None
    if filme_max_meta:
        filme_max_meta_score = f"{filme_max_meta.get_metascore():.0f}"
        filme_max_meta_titulo = filme_max_meta.get_title()
        diretor_popular_meta = filme_max_meta.get_directors()[0] if filme_max_meta.get_directors() else "N/A"


    #A pior série segundo a nota IMDB e a pior série segundo a nota Metascore.
    # Pior Série (IMDB)
    pior_serie_imdb = min(series_com_imdb, key=lambda item_serie: item_serie.get_imdb_rating()) if series_com_imdb else None
    if pior_serie_imdb:
        pior_serie_imdb_titulo = pior_serie_imdb.get_title()
        pior_serie_imdb_rating = f"{pior_serie_imdb.get_imdb_rating():.1f}"

    # Pior Série (Metascore)
    pior_serie_meta = min(series_com_metascore, key=lambda s: s.get_metascore()) if series_com_metascore else None
    if pior_serie_meta:
        pior_serie_meta_titulo = pior_serie_meta.get_title()
        pior_serie_meta_score = f"{pior_serie_meta.get_metascore():.0f}"

    # Filmes "Remake"
    titulos_e_anos = {}
    remakes = set()
    for filme in filmes:
        titulo = filme.get_title()
        ano = filme.get_year()
        if titulo and ano:
            if titulo in titulos_e_anos and ano not in titulos_e_anos[titulo]:
                remakes.add(titulo)
            elif titulo not in titulos_e_anos:
                titulos_e_anos[titulo] = {ano}
            else:
                titulos_e_anos[titulo].add(ano)
            
    lista_remakes = sorted(list(remakes))
    
    # --- Formatação do Relatório ---
    
    relatorio_texto = "================================================\n"
    relatorio_texto += "        RELATÓRIO DE ANÁLISE DISNEY+\n"
    relatorio_texto += "================================================\n\n"

    relatorio_texto += "--- MÉDIAS DE NOTAS E VOTOS (APENAS FILMES) ---\n"
    relatorio_texto += f"Média IMDB de Filmes: {media_imdb:.2f}\n"
    relatorio_texto += f"Média Metascore de Filmes: {media_metascore:.2f}\n"
    relatorio_texto += f"Média de Votos IMDB de Filmes: {media_votos:,.0f} votos\n\n"

    relatorio_texto += "--- TOP/BOTTOM OCORRÊNCIAS ---\n"
    relatorio_texto += f"3 Línguas Mais Usadas: {contagem_linguagens.most_common(3)}\n"
    relatorio_texto += f"3 Línguas Menos Usadas: {contagem_linguagens.most_common()[:-4:-1]}\n\n"
    relatorio_texto += f"3 Atores que Mais Aparecem: {contagem_atores.most_common(3)}\n"
    relatorio_texto += f"3 Atores que Menos Aparecem: {contagem_atores.most_common()[:-4:-1]}\n\n"
    
    relatorio_texto += "--- ANÁLISE DE DIRETORES E LANÇAMENTOS ---\n"
    relatorio_texto += f"Diretor com Mais Filmes: {diretor_mais_filmes[0]} ({diretor_mais_filmes[1]} títulos)\n"
    relatorio_texto += f"Ano com Mais Lançamentos: {ano_mais_filmes[0]} ({ano_mais_filmes[1]} títulos)\n"
    relatorio_texto += f"Diretor do Filme Mais Popular (IMDB {filme_max_imdb_rating}): {diretor_popular_imdb} (Filme: {filme_max_imdb_titulo})\n"
    relatorio_texto += f"Diretor do Filme Mais Popular (Metascore {filme_max_meta_score}): {diretor_popular_meta} (Filme: {filme_max_meta_titulo})\n\n"
    
    relatorio_texto += "--- PIORES SÉRIES ---\n"
    relatorio_texto += f"Pior Série (IMDB): {pior_serie_imdb_titulo} (Nota: {pior_serie_imdb_rating})\n"
    relatorio_texto += f"Pior Série (Metascore): {pior_serie_meta_titulo} (Nota: {pior_serie_meta_score})\n"
    
    relatorio_texto += "\n--- FILMES COM MÚLTIPLOS LANÇAMENTOS (REMAKES/VERSÕES) ---\n"
    if lista_remakes:
        relatorio_texto += "\n".join(lista_remakes)
    else:
        relatorio_texto += "Nenhum filme com múltiplos lançamentos encontrado na amostra."

    # Salvar o relatório em arquivo
    with open(arquivo_relatorio, 'w', encoding='utf-8') as f:
        f.write(relatorio_texto)

    return f"Relatório gerado com sucesso e salvo em '{arquivo_relatorio}'"


# ----------------------------------------------------
#                    Execução Principal
# ----------------------------------------------------

if __name__ == "__main__":
    
    print("\n--- Início ---\n")
    print("\n Disney Data Analysis - Exploring the Magic Behind Disney Shows\n")
    
    # 1. Carregamento e Mapeamento dos Dados
    registros_total = abrir_filmes_disney()
    
    #print (registros_total)

    if registros_total:
        print(f"Total de {len(registros_total)} registros carregados.")
        print(f"Aquivo lido de:'{arquivo_disney}'")
      
        # 2. Geração do Relatório
        relatorio_status = gerar_relatorio(registros_total)
        print(relatorio_status)
        
    print("\n--- Fim ---")




--- Início ---


 Disney Data Analysis - Exploring the Magic Behind Disney Shows

Total de 871 registros carregados.
Aquivo lido de:'./arquivo_origem/disney_plus_shows.csv'
[Filme(Título: 10 Things I Hate About You, Ano: 1999, IMDB: 7.3), Filme(Título: 101 Dalmatians, Ano: 1996, IMDB: 5.7), Filme(Título: 101 Dalmatians 2: Patch's London Adventure, Ano: 2002, IMDB: 5.8), Filme(Título: 102 Dalmatians, Ano: 2000, IMDB: 4.9), Filme(Título: 12 Dates of Christmas, Ano: 2011, IMDB: 6.3), Filme(Título: 20,000 Leagues Under the Sea, Ano: 1954, IMDB: 7.2), Filme(Título: A Bug's Life, Ano: 1998, IMDB: 7.2), Filme(Título: A Celebration of the Music from Coco, Ano: 2020, IMDB: 7.6), Filme(Título: A Goofy Movie, Ano: 1995, IMDB: 6.8), Filme(Título: A Kid in King Arthur's Court, Ano: 1995, IMDB: 4.7), Filme(Título: A Knight for a Day, Ano: 1946, IMDB: 7.0), Filme(Título: A Ring of Endless Light, Ano: 2002, IMDB: 6.1), Filme(Título: A Tale of Two Critters, Ano: 1977, IMDB: 7.1), Filme(Título: A Wrink